<a href="https://colab.research.google.com/github/RogerHeederer/ForAllPytorch/blob/main/seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [114]:
import random
import torch
import torch.nn as nn
from torch import optim
from torchtext.vocab import data

In [188]:
torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [189]:
raw = ["I feel hungry.	나는 배가 고프다.",
       "Pytorch is very easy.	파이토치는 매우 쉽다.",
       "Pytorch is a framework for deep learning.	파이토치는 딥러닝을 위한 프레임워크이다.",
       "Pytorch is very clear to use.	파이토치는 사용하기 매우 직관적이다."]

In [190]:
# 시작과 끝을 알리는 토큰을 고정 지정 해준다.
SOS_token = 0
EOS_token = 1    

In [191]:
# 단어 집합을 만들기 위한 클래스
class Vocab:
    def __init__(self):
        self.vocab2index = {"<SOS>": SOS_token, "<EOS>": EOS_token}
        self.index2vocab = {SOS_token: "<SOS>", EOS_token: "<EOS>"}
        self.vocab_count = {}
        self.n_vocab = len(self.vocab2index)

    def add_vocab(self, sentence):
            for word in sentence.split(" "):
                if word not in self.vocab2index: # 단어집합에 없는 단어이면
                    self.vocab2index[word] = self.n_vocab #해당 단어의 인덱스는 현재 vocab2indx 길이를 계산한 값. 즉 제일 마지막 위치
                    self.vocab_count[word] = 1
                    self.index2vocab[self.n_vocab] = word
                    self.n_vocab += 1
                else:
                    self.vocab_count[word] += 1

In [192]:
# source와 target 데이터를 필터링 해주는 역할
def filter_pair(pair, source_max_length, target_max_length):
    return len(pair[0].split(" ")) < source_max_length and len(pair[1].split(" ")) < target_max_length
    # pair[0]의 길이가 source_max_length보다 작고 and pair[1]의 길이가 target_max_length보다 작으면 True 반환

In [199]:
def preprocess(corpus, source_max_length, target_max_length):
    print("reading corpus...")
    pairs = []
    for line in corpus:
        pairs.append([s for s in line.strip().lower().split("\t")])
    print("Read {} sentence pairs".format(len(pairs)))

    pairs = [pair for pair in pairs if filter_pair(pair, source_max_length, target_max_length)]
    print("Trimmed to {} sentence pairs below".format(len(pairs)))
    print(pairs)

    source_vocab = Vocab()
    target_vocab = Vocab()

    # 여기가 단어집합 완성시키는 로직
    print("Counting words...")
    for pair in pairs:
        source_vocab.add_vocab(pair[0])
        target_vocab.add_vocab(pair[1])

    print("source vocab size =", source_vocab.n_vocab)
    #print(source_vocab.index2vocab)
    return pairs, source_vocab, target_vocab

In [239]:
#convert sentence to the index tensor
def tensorize(vocab, sentence):
    indexes = [vocab.vocab2index[word] for word in sentence.split(" ")]
    indexes.append(vocab.vocab2index["<EOS>"])
    print(indexes)
    print(torch.Tensor(indexes).long().to(device).view(-1,1))
    return torch.Tensor(indexes).long().to(device).view(-1, 1)

중간 테스트 - Preprocess 과정 + Tensorize 과정

지금까지 로직 흐름 정리

1. 코퍼스 읽어서 (영문, 한글) 페어로 1문장씩 짝지어서 리스트화 시킴

2. 영문별, 한글별로 각각 단어,index 페어로 단어집합 만들기

3. i feel hungry -> [2,3,4,1] -> tensor[2,3,4,1]

In [204]:
a,b,c = preprocess(raw, 12, 20)

reading corpus...
Read 4 sentence pairs
Trimmed to 4 sentence pairs below
[['i feel hungry.', '나는 배가 고프다.'], ['pytorch is very easy.', '파이토치는 매우 쉽다.'], ['pytorch is a framework for deep learning.', '파이토치는 딥러닝을 위한 프레임워크이다.'], ['pytorch is very clear to use.', '파이토치는 사용하기 매우 직관적이다.']]
Counting words...
source vocab size = 17
{0: '<SOS>', 1: '<EOS>', 2: 'i', 3: 'feel', 4: 'hungry.', 5: 'pytorch', 6: 'is', 7: 'very', 8: 'easy.', 9: 'a', 10: 'framework', 11: 'for', 12: 'deep', 13: 'learning.', 14: 'clear', 15: 'to', 16: 'use.'}


In [205]:
a

[['i feel hungry.', '나는 배가 고프다.'],
 ['pytorch is very easy.', '파이토치는 매우 쉽다.'],
 ['pytorch is a framework for deep learning.', '파이토치는 딥러닝을 위한 프레임워크이다.'],
 ['pytorch is very clear to use.', '파이토치는 사용하기 매우 직관적이다.']]

In [218]:
b.vocab2index

{'<EOS>': 1,
 '<SOS>': 0,
 'a': 9,
 'clear': 14,
 'deep': 12,
 'easy.': 8,
 'feel': 3,
 'for': 11,
 'framework': 10,
 'hungry.': 4,
 'i': 2,
 'is': 6,
 'learning.': 13,
 'pytorch': 5,
 'to': 15,
 'use.': 16,
 'very': 7}

In [209]:
c.vocab2index

{'<EOS>': 1,
 '<SOS>': 0,
 '고프다.': 4,
 '나는': 2,
 '딥러닝을': 8,
 '매우': 6,
 '배가': 3,
 '사용하기': 11,
 '쉽다.': 7,
 '위한': 9,
 '직관적이다.': 12,
 '파이토치는': 5,
 '프레임워크이다.': 10}

In [235]:
test_tensor = tensorize(b, "i feel hungry.")

[2, 3, 4, 1]
tensor([[2],
        [3],
        [4],
        [1]], device='cuda:0')


In [237]:
# declare simple encoder
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, x, hidden):
        x = self.embedding(x).view(1, 1, -1)
        x, hidden = self.gru(x, hidden)
        return x, hidden

In [238]:
# declare simple decoder
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x, hidden):
        x = self.embedding(x).view(1, 1, -1)
        x, hidden = self.gru(x, hidden)
        x = self.softmax(self.out(x[0]))
        return x, hidden

In [246]:
def train(pairs, source_vocab, target_vocab, encoder, decoder, n_iter, print_every=1000, learning_rate=0.01):
    loss_total = 0

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

    training_batch = [random.choice(pairs) for _ in range(n_iter)]
    training_source = [tensorize(source_vocab, pair[0]) for pair in training_batch]
    training_target = [tensorize(target_vocab, pair[1]) for pair in training_batch]

    criterion = nn.NLLLoss()

    for i in range(1, n_iter + 1):
        source_tensor = training_source[i - 1]
        target_tensor = training_target[i - 1]

        encoder_hidden = torch.zeros([1, 1, encoder.hidden_size]).to(device)

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        source_length = source_tensor.size(0)
        target_length = target_tensor.size(0)

        loss = 0

        for enc_input in range(source_length):
            _, encoder_hidden = encoder(source_tensor[enc_input], encoder_hidden)

        decoder_input = torch.Tensor([[SOS_token]]).long().to(device)
        decoder_hidden = encoder_hidden # connect encoder output to decoder input

        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # teacher forcing

        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        loss_iter = loss.item() / target_length
        loss_total += loss_iter

        if i % print_every == 0:
            loss_avg = loss_total / print_every
            loss_total = 0
            print("[{} - {}%] loss = {:05.4f}".format(i, i / n_iter * 100, loss_avg))

In [256]:
# insert given sentence to check the training
def evaluate(pairs, source_vocab, target_vocab, encoder, decoder, target_max_length):
    for pair in pairs:
        print(">", pair[0])
        print("=", pair[1])
        source_tensor = tensorize(source_vocab, pair[0])
        source_length = source_tensor.size()[0]
        encoder_hidden = torch.zeros([1, 1, encoder.hidden_size]).to(device)

        for ei in range(source_length):
            _, encoder_hidden = encoder(source_tensor[ei], encoder_hidden)

        decoder_input = decoder_input = torch.Tensor([[SOS_token]]).long().to(device)
        decoder_hidden = encoder_hidden
        decoded_words = []

        for di in range(target_max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            _, top_index = decoder_output.data.topk(1)
            if top_index.item() == EOS_token:
                decoded_words.append("<EOS>")
                break
            else:
                decoded_words.append(target_vocab.index2vocab[top_index.item()])

            decoder_input = top_index.squeeze().detach()

        predict_words = decoded_words
        predict_sentence = " ".join(predict_words)
        print("<", predict_sentence)
        print("")

In [248]:
SOURCE_MAX_LENGTH = 10
TARGET_MAX_LENGTH = 12

In [249]:
load_pairs, load_source_vocab, load_target_vocab = preprocess(raw, SOURCE_MAX_LENGTH, TARGET_MAX_LENGTH)

reading corpus...
Read 4 sentence pairs
Trimmed to 4 sentence pairs below
[['i feel hungry.', '나는 배가 고프다.'], ['pytorch is very easy.', '파이토치는 매우 쉽다.'], ['pytorch is a framework for deep learning.', '파이토치는 딥러닝을 위한 프레임워크이다.'], ['pytorch is very clear to use.', '파이토치는 사용하기 매우 직관적이다.']]
Counting words...
source vocab size = 17
{0: '<SOS>', 1: '<EOS>', 2: 'i', 3: 'feel', 4: 'hungry.', 5: 'pytorch', 6: 'is', 7: 'very', 8: 'easy.', 9: 'a', 10: 'framework', 11: 'for', 12: 'deep', 13: 'learning.', 14: 'clear', 15: 'to', 16: 'use.'}


In [250]:
print(random.choice(load_pairs))

['pytorch is very easy.', '파이토치는 매우 쉽다.']


In [251]:
enc_hidden_size = 16
dec_hidden_size = enc_hidden_size
enc = Encoder(load_source_vocab.n_vocab, enc_hidden_size).to(device)
dec = Decoder(dec_hidden_size, load_target_vocab.n_vocab).to(device)

In [252]:
enc

Encoder(
  (embedding): Embedding(17, 16)
  (gru): GRU(16, 16)
)

In [253]:
dec

Decoder(
  (embedding): Embedding(13, 16)
  (gru): GRU(16, 16)
  (out): Linear(in_features=16, out_features=13, bias=True)
  (softmax): LogSoftmax(dim=1)
)

In [ ]:
train(load_pairs, load_source_vocab, load_target_vocab, enc, dec, 5000, print_every=1000)

In [257]:
evaluate(load_pairs, load_source_vocab, load_target_vocab, enc, dec, TARGET_MAX_LENGTH)

> i feel hungry.
= 나는 배가 고프다.
[2, 3, 4, 1]
tensor([[2],
        [3],
        [4],
        [1]], device='cuda:0')
< 나는 배가 고프다. <EOS>

> pytorch is very easy.
= 파이토치는 매우 쉽다.
[5, 6, 7, 8, 1]
tensor([[5],
        [6],
        [7],
        [8],
        [1]], device='cuda:0')
< 파이토치는 매우 쉽다. <EOS>

> pytorch is a framework for deep learning.
= 파이토치는 딥러닝을 위한 프레임워크이다.
[5, 6, 9, 10, 11, 12, 13, 1]
tensor([[ 5],
        [ 6],
        [ 9],
        [10],
        [11],
        [12],
        [13],
        [ 1]], device='cuda:0')
< 파이토치는 딥러닝을 위한 프레임워크이다. <EOS>

> pytorch is very clear to use.
= 파이토치는 사용하기 매우 직관적이다.
[5, 6, 7, 14, 15, 16, 1]
tensor([[ 5],
        [ 6],
        [ 7],
        [14],
        [15],
        [16],
        [ 1]], device='cuda:0')
< 파이토치는 사용하기 매우 직관적이다. <EOS>

